<a href="https://colab.research.google.com/github/Ankityadav000-ux/Brainwave_Matrix_Intern/blob/main/Twitter_Sentiment_Analysis_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#installing kagle library
! pip install kaggle

Upload your Kaggle.json file


In [3]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Importing Twitter Sentiment dataset

In [4]:
# API to fetch the dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [5]:
# extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print ('The dataset is extracted')

The dataset is extracted


Importing the dependencies

In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [9]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='Iso-8859-1')

In [10]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [11]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
# naming the columns and reading the dataset again
column_names = ['Target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names, encoding='Iso-8859-1')

In [13]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [14]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,Target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
Target,0
id,0
date,0
flag,0
user,0
text,0


In [16]:
# checking the distribution of target column
twitter_data['Target'].value_counts()

,count
Target,
0,800000
4,800000


Convert the target "4" to "1"

In [17]:
twitter_data.replace({'Target': {4: 1}}, inplace=True)

In [18]:
# checking the distribution of target column
twitter_data['Target'].value_counts()

,count
Target,
0,800000
1,800000


0-> Negative Tweet
1-> Positive Tweet

Stemming

Stemming is the process of reducing a word to its Root word

example: actor,actress, acting = act

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming (content):
  stemmed_content = re.sub('[^a-zA-z]','', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ''.join(stemmed_content)
  return stemmed_content

In [21]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming) # 50 minutes to completing this execution

In [22]:
twitter_data.head()

,Target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [23]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data['Target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: Target, Length: 1600000, dtype: int64


In [25]:
# seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['Target'].values

In [ ]:
print(X)


['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


Spliting the data to training data and test data


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['abouttowatchsawivanddrinkalilwin' 'hatermagazineimin'
 'eventhoughitsmyfavouritedrinkithinkitsthevodkaandcokethatwipesmymindallthetimethinkimgonnahavetofindanewdrink'
 ... 'iseagerformondayafternoon'
 'hopeeveryoneandtheirmotherhadagreatdaycantwaittohearwhattheguyshaveinstoretomorrow'
 'ilovewakinguptofolgerstoobadmyvoicewasdeeperthanhi']


In [28]:
print(X_test)

['mmangenmdoingfineihaventhadmuchtimetochatontwitterhubbyisbackforthesummeramptendstodominatemyfreetim'
 'atahsmayshowwruthkimampgeoffreysanhueza'
 'ishataramaybeitwasonlyabayareathangdammit' ...
 'destinineverthelesshoorayformembersandhaveawonderfulandsafetrip'
 'notfeelingtoowel' 'supersandrothanky']


In [29]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [30]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1286678 stored elements and shape (1280000, 1260418)>
  Coords	Values
  (0, 7869)	1.0
  (1, 374026)	1.0
  (2, 263265)	1.0
  (3, 548840)	1.0
  (4, 1131050)	1.0
  (5, 186521)	1.0
  (6, 660150)	1.0
  (7, 431866)	1.0
  (8, 375854)	1.0
  (9, 418157)	1.0
  (10, 457155)	1.0
  (11, 950994)	1.0
  (12, 1170819)	1.0
  (13, 338356)	1.0
  (14, 1034428)	1.0
  (15, 1257130)	1.0
  (16, 30247)	1.0
  (17, 624890)	1.0
  (18, 374604)	1.0
  (19, 169968)	1.0
  (20, 944314)	1.0
  (21, 396327)	1.0
  (22, 291778)	1.0
  (23, 206184)	1.0
  (24, 1259240)	1.0
  :	:
  (1279976, 1116076)	1.0
  (1279977, 943617)	1.0
  (1279978, 719802)	1.0
  (1279979, 805329)	1.0
  (1279980, 869980)	1.0
  (1279981, 591760)	1.0
  (1279982, 957620)	1.0
  (1279983, 416619)	1.0
  (1279984, 85330)	1.0
  (1279985, 62659)	1.0
  (1279986, 1020839)	1.0
  (1279987, 781489)	1.0
  (1279988, 1224379)	1.0
  (1279989, 454919)	1.0
  (1279990, 469151)	1.0
  (1279991, 252736)	1.0
  (1279992

In [31]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8896 stored elements and shape (320000, 1260418)>
  Coords	Values
  (22, 141423)	1.0
  (36, 794874)	1.0
  (55, 1237902)	1.0
  (104, 810608)	1.0
  (316, 679477)	1.0
  (317, 186544)	1.0
  (326, 535094)	1.0
  (343, 487353)	1.0
  (356, 1009575)	1.0
  (405, 297462)	1.0
  (411, 759103)	1.0
  (412, 675932)	1.0
  (503, 927289)	1.0
  (575, 488487)	1.0
  (576, 71801)	1.0
  (585, 558000)	1.0
  (597, 441891)	1.0
  (704, 441375)	1.0
  (730, 526717)	1.0
  (735, 1196033)	1.0
  (763, 205037)	1.0
  (851, 234118)	1.0
  (887, 950013)	1.0
  (905, 1086508)	1.0
  (965, 67598)	1.0
  :	:
  (319388, 473709)	1.0
  (319391, 366994)	1.0
  (319401, 72879)	1.0
  (319403, 1015019)	1.0
  (319419, 439721)	1.0
  (319442, 1159491)	1.0
  (319462, 335454)	1.0
  (319474, 366802)	1.0
  (319550, 1025070)	1.0
  (319573, 1093216)	1.0
  (319595, 380676)	1.0
  (319630, 420045)	1.0
  (319632, 328028)	1.0
  (319640, 337269)	1.0
  (319649, 569032)	1.0
  (319673, 478004)	

Training the Machine Learning Model

Logistic Regression

In [32]:
model = LogisticRegression(max_iter=1000)

In [34]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score

In [35]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [37]:
print('Accuracy score on the training data:', training_data_accuracy)

Accuracy score on the training data: 0.9980109375


In [40]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [42]:
print('Accuracy score on the test data:', test_data_accuracy)


Accuracy score on the test data: 0.512775


Model accuracy =51.2%

Saving the trained model

In [43]:
import pickle

In [44]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions

In [45]:
# loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [49]:

X_new = X_test[200]
print(Y_test[200])
prediction = loaded_model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')


1
[1]
Positive Tweet


In [53]:

X_new = X_test[100]
print(Y_test[100])
prediction = loaded_model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')


0
[1]
Positive Tweet
